In [ ]:
# instalação dos pacotes necessários na versão mais nova
%pip install -U scikit-learn pandas numpy seaborn

In [ ]:
# importando bibliotecas necessárias
import pandas as pd
import numpy as np

# graphs
import matplotlib.pyplot as plt
import seaborn as sns

# separação de dados
from sklearn.model_selection import train_test_split

# padronização dos dados
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# construção de filtro para ColumnTransformer
from sklearn.compose import make_column_selector

# criação de pipeline para tratamento do conjunto de dados
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

# modelo
from sklearn.linear_model import LassoCV, RidgeCV

In [ ]:
# agora uma função que vai printar bonitinho as métricas
def print_metrics(model: Pipeline, model_name: str, X_test: np.ndarray, y_test: np.ndarray, y_pred: np.ndarray) -> str:
  """
  Returns the Mean Absolute Error (MAE), Mean Squared Error (MSE), Root Mean Squared Error (RMSE) and R-squared scores of the model
      
      Arguments:
          model_name {str} -- name of the model
          y_test {array} -- array of the test labels
          y_pred {array} -- array of the predicted labels
          y_test_pca {array} -- array of the test labels after PCA
          y_pred_pca {array} -- array of the predicted labels after PCA
      
      Returns:
          A formatted string containing the accuracy, precision and recall scores of the model for both non-PCA and PCA.
  
  """
  #imports
  from sklearn.metrics import mean_absolute_error, mean_squared_error

  return (f'''

Mean Absolute Error (MAE) do Modelo {model_name}: {mean_absolute_error(y_test, y_pred):.3f}

Mean Squared Error (MSE) do Modelo {model_name}: {mean_squared_error(y_test, y_pred, squared=True):.3f}

Root Mean Squared Error (RMSE) do Modelo {model_name}: {mean_squared_error(y_test, y_pred, squared=False):.3f}

R-squared do Modelo {model_name}: {model.score(X_test, y_test):.3f}
''')

In [71]:
def get_features(model: Pipeline, model_name: str, ispca: bool) -> str:
  """
  Get Features Function
  
  Returns a string containing the number of original features, the number of features selected by the model (in case the model is a Lasso) and the name of the features selected (will return all features if it's not a Lasso).
      
      Arguments:

          - model {Pipeline} -- a pipeline object containing an estimator/model with fit/transform/predict operations.

          - model_name {str} -- name of the model used inside the pipeline to identify the model

          - ispca {bool} -- if the model passed is a model with pca, this should be true, otherwise it should be false.
      
      Returns:
      
          Returns a string containing the number of original features, the number of features selected by the model (in case the model is a Lasso) and the name of the features selected (will return all features if it's not a Lasso).
  
  """

  
  preprocessor_fitted = model['preprocessor']
  
  if ispca:
    feature_names = (
      list(preprocessor_fitted.named_transformers_['num']['pca'].get_feature_names_out()) +
      list(preprocessor_fitted.named_transformers_['cat']['pca'].get_feature_names_out())
      )
  
  else:
    feature_names = (
      list(preprocessor_fitted.named_transformers_['num']['imputer'].get_feature_names_out()) +
      list(preprocessor_fitted.named_transformers_['cat']['one_hot_encoder'].get_feature_names_out())
      )
  feature_names = np.array(feature_names)
  kept = ~np.isclose(model[model_name].coef_, 0)
  
  if 'lasso' not in model_name:
    return(f'''
    Atenção: O modelo passado não é Lasso, logo todas as variáveis passadas terão sido selecionadas.
    Número de Features Original: {len(feature_names)}
    Número de Features selecionadas pelo {model_name}: {len(feature_names[kept])}
    Features Selecionadas:
    {feature_names[kept].tolist()}
    ''')

  return(f'''
    Número de Features Original: {len(feature_names)}
    Número de Features selecionadas pelo {model_name}: {len(feature_names[kept])}
    Features Selecionadas:
    {feature_names[kept].tolist()}
    ''')

In [ ]:
from warnings import simplefilter
#from sklearn.exceptions import UserWarning

simplefilter("ignore", category=UserWarning)

## 1. Análise exploratória de dados (1.0 ponto)

Realize a análise exploratória de dados. Observe cada variável, como se comportam em relação a outras variáveis independentes e com a variável dependente.


In [47]:
# carregando dataset de teste

dataframe = pd.read_csv('dado/train.csv')
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [77]:
# Separando dados de treinamento do modelo
X = dataframe.drop(['Id', 'SalePrice'], axis=1)
y = dataframe.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. Levantamento de hipóteses (1.0 ponto)

Descreva quais hipóteses você observou ao fazer a análise exploratória de dados.



O conjunto de dados possui 79 features para prever um target de valor contínuo, logo é preciso de uma regressão linear. Ao invés de separar os dados, optei por automatizar o processo usando pipeline e deixando que o Lasso escolha as melhores variáveis.

## 3. Apresentação das ideias obtidas (1.0 ponto)

Apresente com gráficos as suas observações e a descreva cada gráfico.


## 4. Preparação dos dados (1.0 ponto)

Faça um processamento nos dados, preenchendo valores faltantes, removendo dados ou variáveis inconsistentes e normalizado os dados



#### Passos

- Encoding -> `OneHotEncoder`
- Impute Missing Values -> `SimpleImputer`
- Scaler -> `StandardScaler`
- PCA -> `PCA`
- Model -> `LassoCV`
- Cross-Val -> _Interno no_ `LassoCV`

O `OneHotEncoder` é utilizado para codificar as variáveis categóricas em variáveis numéricas (mas ainda mantendo a propriedade de categoria, digamos assim). O seu funcionamento é parecido com o método `get_dummies` do Pandas, porém mais simples de ser aplicado e é possível utilizá-lo dentro de um `ColumnTransformer` que, por sua vez, é usado num `Pipeline` para automatizar as tarefas de pré-processamento e treinamento do modelo. Outra opção seria utilizar o `OrdinalEncoder` porém a própria documentação aponta que o mesmo pode afetar o desempenho de modelos lineares, tais como o LassoCV e o RidgeCV.

In [78]:
# filtros para o pipeline

col_num = make_column_selector(dtype_include=np.number)
col_tex = make_column_selector(dtype_include=object)

# construindo pipeline para o modelo de regressão usando LassoCV sem pca
## numeric_transformer, categorical_transformer e preprocessor podem ser reutilizados em outros pipelines sem pca

numeric_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
        ]
)

categorical_transformer = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
        ]
)

preprocessor = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer, col_num),
    ("cat", categorical_transformer, col_tex)
  ]
)

modelLassoCV = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff91d876c50>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff91d876710>)])),
                ('lassocv', LassoCV(cv=5))])

In [79]:
# fit e predict do modelo
modelLassoCV.fit(X_train, y_train)
y_pred = modelLassoCV.predict(X_test)

In [80]:
print(get_features(model=modelLassoCV, model_name='lassocv', ispca= False))


    Número de Features Original: 299
    Número de Features selecionadas pelo lassocv: 90
    Features Selecionadas:
    ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'x0_RM', 'x3_IR2', 'x4_Bnk', 'x4_HLS', 'x6_CulDSac', 'x7_Mod', 'x8_BrkSide', 'x8_Crawfor', 'x8_Edwards', 'x8_Gilbert', 'x8_Mitchel', 'x8_NAmes', 'x8_NWAmes', 'x8_NoRidge', 'x8_NridgHt', 'x8_Somerst', 'x8_StoneBr', 'x9_Norm', 'x10_PosN', 'x11_1Fam', 'x11_2fmCon', 'x12_1Story', 'x12_2Story', 'x13_Gable', 'x14_ClyTile', 'x15_BrkFace', 'x15_HdBoard', 'x16_ImStucc', 'x16_Plywood', 'x16_Wd Sdng', 'x17_BrkFace', 'x17_None', 'x18_Ex'

In [81]:
# pontuações
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV, model_name="Pipeline LassoCV", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV: 17498.255

Mean Squared Error (MSE) do Modelo Pipeline LassoCV: 865315089.022

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV: 29416.239

R-squared do Modelo Pipeline LassoCV: 0.887



## 5. Criação e seleção de características (1.0 ponto)

Observando as variáveis disponíveis, é possível criar novas variáveis? Ou trazer de um dado externo novos dados para agregar aos existentes? É possível selecionar uma variável ou grupo de variáveis que melhor explica a variável dependente?


Aqui eu faço pipeline com pca

In [82]:
# construindo pipeline para o modelo de regressão usando LassoCV com pca
## numeric_transformer_pca, categorical_transformer_pca e preprocessor_pca podem ser reutilizados em outros pipelines com pca

numeric_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="median")),
          ("scaler", StandardScaler()),
          ("pca", PCA(n_components='mle'))
        ]
)

categorical_transformer_pca = Pipeline(
  steps=[
          ("imputer", SimpleImputer(strategy="constant", fill_value="0")),
          ("one_hot_encoder", OneHotEncoder(dtype=int, handle_unknown='ignore', drop='if_binary', sparse=False)),
          ("pca", PCA(n_components='mle'))
        ]
)

preprocessor_pca = ColumnTransformer(
  transformers=[
    ("num", numeric_transformer_pca, col_num),
    ("cat", categorical_transformer_pca, col_tex)
  ]
)

modelLassoCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor_pca), 
    ("lassocv", LassoCV(cv=5))
  ]
)

modelLassoCV_pca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff91d876c50>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff91d876710>)])),
                ('lassocv', LassoCV(cv=5))])

In [83]:
# fit e predict do modelo LassoCV com pca
modelLassoCV_pca.fit(X_train, y_train)
y_pred = modelLassoCV_pca.predict(X_test)

In [84]:
print(get_features(model=modelLassoCV_pca, model_name='lassocv', ispca= True))


    Número de Features Original: 249
    Número de Features selecionadas pelo lassocv: 105
    Features Selecionadas:
    ['pca0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5', 'pca6', 'pca7', 'pca8', 'pca9', 'pca10', 'pca11', 'pca13', 'pca15', 'pca16', 'pca17', 'pca18', 'pca19', 'pca21', 'pca22', 'pca23', 'pca24', 'pca25', 'pca26', 'pca27', 'pca28', 'pca30', 'pca31', 'pca32', 'pca33', 'pca0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5', 'pca6', 'pca7', 'pca8', 'pca9', 'pca10', 'pca12', 'pca13', 'pca14', 'pca15', 'pca16', 'pca17', 'pca18', 'pca19', 'pca20', 'pca22', 'pca23', 'pca24', 'pca25', 'pca26', 'pca27', 'pca29', 'pca32', 'pca33', 'pca34', 'pca35', 'pca36', 'pca39', 'pca40', 'pca41', 'pca42', 'pca43', 'pca44', 'pca45', 'pca47', 'pca48', 'pca51', 'pca52', 'pca53', 'pca55', 'pca56', 'pca57', 'pca59', 'pca60', 'pca61', 'pca62', 'pca64', 'pca67', 'pca68', 'pca69', 'pca71', 'pca75', 'pca76', 'pca78', 'pca82', 'pca85', 'pca86', 'pca88', 'pca89', 'pca94', 'pca96', 'pca98', 'pca99', 'pca102', 'pca

In [85]:
# pontuações
print(f'Pontuação do Modelo: {print_metrics(model=modelLassoCV_pca, model_name="Pipeline LassoCV com PCA", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV com PCA: 19116.457

Mean Squared Error (MSE) do Modelo Pipeline LassoCV com PCA: 1000708924.386

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV com PCA: 31633.984

R-squared do Modelo Pipeline LassoCV com PCA: 0.870



### Testando o Pipeline com RidgeCV com e sem PCA

In [86]:
# construindo pipeline para o modelo de regressão usando RidgeCV com PCA

modelRidgeCV_pca = Pipeline(
  steps=[
    ("preprocessor", preprocessor_pca), 
    ("ridgecv", RidgeCV(cv=5))
  ]
)

modelRidgeCV_pca

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff91d876c50>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('pca',
                                                                   PCA(n_components='mle'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff91d876710>)])),
                ('ridgecv', RidgeCV(cv=5))])

In [87]:
# fit e predict do modelo RidgeCV com pca
modelRidgeCV_pca.fit(X_train, y_train)
y_pred = modelRidgeCV_pca.predict(X_test)

In [88]:
print(get_features(model=modelRidgeCV_pca, model_name='ridgecv', ispca= True))


    Atenção: O modelo passado não é Lasso, logo todas as variáveis passadas terão sido selecionadas.
    Número de Features Original: 249
    Número de Features selecionadas pelo ridgecv: 249
    Features Selecionadas:
    ['pca0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5', 'pca6', 'pca7', 'pca8', 'pca9', 'pca10', 'pca11', 'pca12', 'pca13', 'pca14', 'pca15', 'pca16', 'pca17', 'pca18', 'pca19', 'pca20', 'pca21', 'pca22', 'pca23', 'pca24', 'pca25', 'pca26', 'pca27', 'pca28', 'pca29', 'pca30', 'pca31', 'pca32', 'pca33', 'pca0', 'pca1', 'pca2', 'pca3', 'pca4', 'pca5', 'pca6', 'pca7', 'pca8', 'pca9', 'pca10', 'pca11', 'pca12', 'pca13', 'pca14', 'pca15', 'pca16', 'pca17', 'pca18', 'pca19', 'pca20', 'pca21', 'pca22', 'pca23', 'pca24', 'pca25', 'pca26', 'pca27', 'pca28', 'pca29', 'pca30', 'pca31', 'pca32', 'pca33', 'pca34', 'pca35', 'pca36', 'pca37', 'pca38', 'pca39', 'pca40', 'pca41', 'pca42', 'pca43', 'pca44', 'pca45', 'pca46', 'pca47', 'pca48', 'pca49', 'pca50', 'pca51', 'pca52', 'pca53', 'pc

In [89]:
# pontuações
print(f'Pontuação do Modelo: {print_metrics(model=modelRidgeCV_pca, model_name="Pipeline RidgeCV com PCA", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline RidgeCV com PCA: 19194.234

Mean Squared Error (MSE) do Modelo Pipeline RidgeCV com PCA: 933839816.113

Root Mean Squared Error (RMSE) do Modelo Pipeline RidgeCV com PCA: 30558.793

R-squared do Modelo Pipeline RidgeCV com PCA: 0.878



In [90]:
# construindo pipeline para o modelo de regressão usando RidgeCV sem PCA

modelRidgeCV = Pipeline(
  steps=[
    ("preprocessor", preprocessor), 
    ("ridgecv", RidgeCV(cv=5))
  ]
)

modelRidgeCV

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff91d876c50>),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='0',
                                                                                 strategy='constant')),
                                                                  ('one_hot_encoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ff91d876710>)])),
                ('ridgecv', RidgeCV(cv=5))])

In [91]:
# fit e predict do modelo RidgeCV sem pca
modelRidgeCV.fit(X_train, y_train)
y_pred = modelRidgeCV.predict(X_test)

In [92]:
print(get_features(model=modelRidgeCV, model_name='ridgecv', ispca= False))


    Atenção: O modelo passado não é Lasso, logo todas as variáveis passadas terão sido selecionadas.
    Número de Features Original: 299
    Número de Features selecionadas pelo ridgecv: 299
    Features Selecionadas:
    ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'x0_C (all)', 'x0_FV', 'x0_RH', 'x0_RL', 'x0_RM', 'x1_Pave', 'x2_0', 'x2_Grvl', 'x2_Pave', 'x3_IR1', 'x3_IR2', 'x3_IR3', 'x3_Reg', 'x4_Bnk', 'x4_HLS', 'x4_Low', 'x4_Lvl', 'x5_NoSeWa', 'x6_Corner', 'x6_CulDSac', 'x6_FR2', 'x6_FR3', 'x6_Inside', 'x7_Gtl', 'x7_Mod', 'x7_Sev', '

In [93]:
# pontuações
print(f'Pontuação do Modelo: {print_metrics(model=modelRidgeCV, model_name="Pipeline RidgeCV sem PCA", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo: 

Mean Absolute Error (MAE) do Modelo Pipeline RidgeCV sem PCA: 19194.234

Mean Squared Error (MSE) do Modelo Pipeline RidgeCV sem PCA: 933839816.113

Root Mean Squared Error (RMSE) do Modelo Pipeline RidgeCV sem PCA: 30558.793

R-squared do Modelo Pipeline RidgeCV sem PCA: 0.878



## 6. Decisão do modelo (1.0 ponto)

Dada toda a análise realizada nos pontos anteriores, qual modelo melhor se adequa a este problema? E por quê?



Com base nos resultados encontrados, eu escolheria o modelo original `modelLassoCV` por ter definido um número menor de features (90) e ainda reter um score de R² de 0.887. O Lasso testado com PCA acabou incluíndo 108 variáveis e levou mais tempo para fazer o fit, logo apresentando um custo computacional maior. O modelo com Ridge apresentou um R² muito próximo do `modelLassoCV` porém usa todas as features do dataset de PCA (300 features no dataset sem pca (Lasso usou 90) X 250 features no dataset com pca). É importante notar que a propriedade estatística de "escolher" features é exclusiva do Lasso por usar um cálculo de pesos que classifica algumas variáveis com peso 0, dessa forma podemos dizer que ele "escolhe" as principais variáveis, não necessitando de um RFE.

Entretanto, após testes locais com `modelRidgeCV`, chego a conclusão de que, mesmo usando 300 features o Ridge será o melhor modelo uma vez que é mais barato computacionalmente (inclusive levou apenas 1.1s para o fit) do que o LassoCV nesse caso em específico. Entretanto, se o conjunto de dados fosse maior, talvez o Lasso fosse a melhor escolha por diminuir a quantidade de features utilizadas e gerar um modelo mais "simples".

Conclusão: modelRidgeCV

## 7. Modelagem (1.0 ponto)

Divida os dados em teste e treino, construa o seu modelo e teste alterar os parâmetros utilizados por ele.



Okay, aqui meu notebook fica um pouco diferente do esperado. Como montei com Pipeline, o modelo já foi treinado lá no topo. Vou apenas imprimir os resultados novamente e testar com o conjunto de testes.

In [94]:
print(f'Pontuação do Modelo LassoCV sem PCA: {print_metrics(model=modelLassoCV_pca, model_name="Pipeline LassoCV sem PCA", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo LassoCV sem PCA: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV sem PCA: 19194.234

Mean Squared Error (MSE) do Modelo Pipeline LassoCV sem PCA: 933839816.113

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV sem PCA: 30558.793

R-squared do Modelo Pipeline LassoCV sem PCA: 0.870



In [95]:
print(f'Pontuação do Modelo LassoCV com PCA: {print_metrics(model=modelLassoCV, model_name="Pipeline LassoCV com PCA", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo LassoCV com PCA: 

Mean Absolute Error (MAE) do Modelo Pipeline LassoCV com PCA: 19194.234

Mean Squared Error (MSE) do Modelo Pipeline LassoCV com PCA: 933839816.113

Root Mean Squared Error (RMSE) do Modelo Pipeline LassoCV com PCA: 30558.793

R-squared do Modelo Pipeline LassoCV com PCA: 0.887



In [96]:
print(f'Pontuação do Modelo RidgeCV com PCA: {print_metrics(model=modelRidgeCV_pca, model_name="Pipeline RidgeCV com PCA", X_test=X_test, y_test=y_test, y_pred=y_pred)}')

Pontuação do Modelo RidgeCV com PCA: 

Mean Absolute Error (MAE) do Modelo Pipeline RidgeCV com PCA: 19194.234

Mean Squared Error (MSE) do Modelo Pipeline RidgeCV com PCA: 933839816.113

Root Mean Squared Error (RMSE) do Modelo Pipeline RidgeCV com PCA: 30558.793

R-squared do Modelo Pipeline RidgeCV com PCA: 0.878



## 8. Histórico (1.0 pontos)

Preencha a tabela abaixo com cada teste realizado com seu modelo. Não se preocupe caso voce mude de ideia sobre qual modelo testar, mas não deixe de testá-lo exaustivamente e, caso precise, retorne para o passo de criação e seleção de características.


| Modelo | Parâmetros | MAE | R² | Observação |
| ------ | ---------- | --- | -- | ---------- |
| LassoCV sem PCA | Escolhidos pelo Lasso, Cross-Validation: 5 dobras | 19171.747 | 0.872 | Modelo em Pipeline, usando LassoCV sem PCA. Usou 89 das 299 features disponíveis. Tempo de Fit: 1.9s  |
| LassoCV com PCA | Escolhidos pelo Lasso, Cross-Validation: 5 dobras | 19171.747 | 0.887 | Modelo em Pipeline, usando LassoCV com PCA. Usou 107 das 249 features disponíveis. Tempo de Fit: 7.2s  |
| RidgeCV com PCA | Todos as features vindas do PCA, Cross-Validation: 5 dobras | 19171.747 | 0.887 | Modelo em Pipeline, usando RidgeCV com PCA. Usou 249 das 249 features disponíveis. Tempo de Fit: 7.5s  |
| **RidgeCV sem PCA** | Todos as features do conjunto de dados, Cross-Validation: 5 dobras | 19171.747 | 0.878 | Modelo em Pipeline, usando RidgeCV com PCA. Usou 299 das 299 features disponíveis. Tempo de Fit: 1.1s  |

## 9. Resultado (2.0 pontos)

Reúna os seus achados, os valores das métricas do seu melhor modelo obtido e o melhor conjunto de variáveis. Escreva abaixo um relatório que seria enviado para uma diretoria de uma empresa sobre seu trabalho.


Conforme os dados que seguem em anexo, optou-se pela escolha do Pipeline RidgeCV sem PCA para ser enviado à produção. O Pipeline, além de integrar as tarefas de pré-processamento dos dados (economizando tempo dos nossos funcionários), possui um custo computacional menor do que os outros pipelines testados. 

Ao invés dos nossos funcionários precisarem tratar manualmente os novos conjuntos de dados para só depois enviar para o modelo, o pipeline faz tudo isso de forma automática seguindo as regras estabelicidas e descartando quaisquer dados que não correspondam a uma regra, evitando que dados extras causem ruído no modelo ou mesmo que quebrem o deploy.

É importante notar também que, caso a empresa consiga mais dados para construirmos novas features no conjunto de dados, será necessária uma nova comparação do custo computacional entre Lasso e Ridge. Isso acontece porque mesmo o Ridge sendo mais barato, o Lasso tem a vantagem de diminuir o peso de várias features para 0, retornando um modelo mais simples e que pode acabar sendo mais rápido e barato para o deploy.